In [1]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.

In [2]:
excel_path = "/Users/AlexanderLarsson/documents/VSCode/test_indata.xlsx"
indata_sheet = "test_case_1_whNormal_NBD"
indataDF = pd.read_excel(excel_path,indata_sheet)
outdataDF = indataDF.copy()
indataDF 

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev
0,1,RDC,Johannesburg,1,40,1,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,1,4,1,0.95,Poisson,1.0,NaN
2,3,Dealer,Deal2,3,25,1,0.95,Poisson,1.0,NaN
3,4,Dealer,Deal3,2,11,1,0.95,Poisson,1.0,NaN
4,5,Dealer,Deal4,1,37,1,0.95,Poisson,1.0,NaN
5,6,Dealer,Deal5,4,1,1,0.95,Poisson,1.0,NaN
6,7,Dealer,Deal6,3,3,1,0.95,Poisson,1.0,NaN
7,8,Dealer,Deal7,2,8,1,0.95,Poisson,1.0,NaN
8,9,Dealer,Deal8,5,45,1,0.95,Poisson,1.0,NaN
9,10,Dealer,Deal9,4,9,1,0.95,Poisson,1.0,NaN


In [3]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()

for id in indataDF.get(indataDF["Type"] == "Dealer").get("Installation id"):
    if indataDF.get(indataDF["Installation id"] == id).get("Demand type").to_numpy() == "Poisson":
        indataDF.loc[indataDF["Installation id"] == id, "Demand stdev"] = math.sqrt(indataDF.loc[outdataDF["Installation id"]== id, "Demand mean"].to_numpy())

# New change

sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()


Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))

In [4]:

rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

rdc_f_u_probability_array = f_nq[:-1] #CHANGE HERE


outdataDF.loc[outdataDF["Type"] == "Dealer","Q, subbatches"] = Q_dealer_arr/Q_subbatch_size

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC", "Demand variance"] = sigma2_L * Q_subbatch_size/L_wh

outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,"Q, subbatches",Lead time demand mean,Lead time demand stdev,Demand variance
0,1,RDC,Johannesburg,1,40,1,NaN,NBD,10.0,2.054649,NaN,100.0,20.546486,4.221581
1,2,Dealer,Deal1,1,4,1,0.95,Poisson,1.0,NaN,4.0,NaN,NaN,NaN
2,3,Dealer,Deal2,3,25,1,0.95,Poisson,1.0,NaN,25.0,NaN,NaN,NaN
3,4,Dealer,Deal3,2,11,1,0.95,Poisson,1.0,NaN,11.0,NaN,NaN,NaN
4,5,Dealer,Deal4,1,37,1,0.95,Poisson,1.0,NaN,37.0,NaN,NaN,NaN
5,6,Dealer,Deal5,4,1,1,0.95,Poisson,1.0,NaN,1.0,NaN,NaN,NaN
6,7,Dealer,Deal6,3,3,1,0.95,Poisson,1.0,NaN,3.0,NaN,NaN,NaN
7,8,Dealer,Deal7,2,8,1,0.95,Poisson,1.0,NaN,8.0,NaN,NaN,NaN
8,9,Dealer,Deal8,5,45,1,0.95,Poisson,1.0,NaN,45.0,NaN,NaN,NaN
9,10,Dealer,Deal9,4,9,1,0.95,Poisson,1.0,NaN,9.0,NaN,NaN,NaN


RDC reorder-point optimization

In [5]:
h_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Holding cost").to_numpy()
fill_rate_target_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Target item fill rate").to_numpy()
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
l_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Transport time").to_numpy()
mu_wh = mu_L/L_wh * Q_subbatch_size

beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,mu_dealer_arr,sigma_dealer_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))

beta_arr = np.array(beta_list)

beta_rdc = weighting_backorder_cost(mu_dealer_arr,mu_wh,beta_arr)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Shortage cost"] = p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "RDC", "Beta"] = beta_rdc
outdataDF.loc[outdataDF["Type"] == "Dealer", "Beta"] = beta_arr
print(f"Optimal weighted induced backorder cost at the warehouse is: {beta_rdc}, betas are: {beta_arr}")



Optimal weighted induced backorder cost at the warehouse is: 0.4304258274108533, betas are: [0.7865432  0.16102878 0.33212106 0.09337052 0.7030854  0.63211749
 0.45059161 0.09921657 0.447933   0.59825065]


In [6]:
h_rdc = float(indataDF.get(indataDF["Type"] == "RDC").get("Holding cost"))
Q_0 = int(int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))/Q_subbatch_size)

R_0 = warehouse_optimization(Q_subbatch_size,Q_0,rdc_f_u_probability_array,h_rdc,beta_rdc)

outdataDF.loc[outdataDF["Type"] == "RDC", "Q, subbatches"] = Q_0
outdataDF.loc[outdataDF["Type"] == "RDC", "R, subbatches"] = R_0
outdataDF


Starting optimizing, R = 0, c = 57.96325362303906, c+1 = 50.25604959609621
Doing upwards optimizing, R = 1, c = 50.25604959609621, c+1 = 42.61195152612072
Doing upwards optimizing, R = 2, c = 42.61195152612072, c+1 = 35.16334905037031
Doing upwards optimizing, R = 3, c = 35.16334905037031, c+1 = 28.20700205861912
Doing upwards optimizing, R = 4, c = 28.20700205861912, c+1 = 22.26540869517459
Doing upwards optimizing, R = 5, c = 22.26540869517459, c+1 = 18.05054784200384
Doing upwards optimizing, R = 6, c = 18.05054784200384, c+1 = 16.281640849778515
Doing upwards optimizing, R = 7, c = 16.281640849778515, c+1 = 17.417749208669427


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,"Q, subbatches",Lead time demand mean,Lead time demand stdev,Demand variance,Shortage cost,Beta,"R, subbatches"
0,1,RDC,Johannesburg,1,40,1,NaN,NBD,10.0,2.054649,4.0,100.0,20.546486,4.221581,NaN,0.430426,-20.0
1,2,Dealer,Deal1,1,4,1,0.95,Poisson,1.0,NaN,4.0,NaN,NaN,NaN,19.0,0.786543,NaN
2,3,Dealer,Deal2,3,25,1,0.95,Poisson,1.0,NaN,25.0,NaN,NaN,NaN,19.0,0.161029,NaN
3,4,Dealer,Deal3,2,11,1,0.95,Poisson,1.0,NaN,11.0,NaN,NaN,NaN,19.0,0.332121,NaN
4,5,Dealer,Deal4,1,37,1,0.95,Poisson,1.0,NaN,37.0,NaN,NaN,NaN,19.0,0.093371,NaN
5,6,Dealer,Deal5,4,1,1,0.95,Poisson,1.0,NaN,1.0,NaN,NaN,NaN,19.0,0.703085,NaN
6,7,Dealer,Deal6,3,3,1,0.95,Poisson,1.0,NaN,3.0,NaN,NaN,NaN,19.0,0.632117,NaN
7,8,Dealer,Deal7,2,8,1,0.95,Poisson,1.0,NaN,8.0,NaN,NaN,NaN,19.0,0.450592,NaN
8,9,Dealer,Deal8,5,45,1,0.95,Poisson,1.0,NaN,45.0,NaN,NaN,NaN,19.0,0.099217,NaN
9,10,Dealer,Deal9,4,9,1,0.95,Poisson,1.0,NaN,9.0,NaN,NaN,NaN,19.0,0.447933,NaN


In [7]:
W = waiting_time(negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] =outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev").to_numpy()
outdataDF.loc[outdataDF["Type"] == "RDC", "Lead time"] = indataDF.get(indataDF["Type"] == "RDC").get("Transport time")

In [8]:
opt_dealer_list = []
L_dealer_arr = outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time")
for Q,L_est,fill_rate_target,demand_type,mu,sigma in zip(Q_dealer_arr,L_dealer_arr,fill_rate_target_arr,demand_type_arr, mu_dealer_arr,sigma_dealer_arr):
    print(demand_type, mu, sigma)
    opt_dealer_list.append(dealer_R_optimization(Q,L_est,fill_rate_target,demand_type,mu,demand_variance = math.pow(sigma,2)))

R_opt_dealer_list,fill_rate_dealer_list,exp_stock_on_hand_list = [],[],[]
for tup in opt_dealer_list:
    R_opt_dealer_list.append(tup[0])
    fill_rate_dealer_list.append(tup[2])
    exp_stock_on_hand_list.append(tup[3])

outdataDF.loc[outdataDF["Type"] == "Dealer", "R optimal"] = R_opt_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Realized item fill rate"] = fill_rate_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected stock on hand"] = exp_stock_on_hand_list

outdataDF.loc[outdataDF["Type"] == "RDC","R optimal"] = R_0*Q_subbatch_size # In units.
outdataDF.loc[outdataDF["Type"] == "RDC","Expected stock on hand"] = positive_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)
outdataDF.loc[outdataDF["Type"] == "RDC","Expected backorders"] = negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)


Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0
Poisson 1.0 1.0


In [9]:
# List comprehension for exp backorders.
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected backorders"] = [ 
    expected_backorders_discrete(R,Q,lt_mu,exp_stock_on_hand) for 
    R,Q,lt_mu,exp_stock_on_hand in zip(R_opt_dealer_list,Q_dealer_arr,
    outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time demand mean").to_numpy(),
    exp_stock_on_hand_list) ]

In [10]:
# Adding costs
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected holding costs per time unit"] = h_dealer_arr*np.array(exp_stock_on_hand_list)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected shortage costs per time unit"] = outdataDF.get(
    outdataDF["Type"] == "Dealer").get("Expected backorders").to_numpy()*p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Total expected costs"] = outdataDF.get(outdataDF["Type"] == "Dealer").get("Expected holding costs per time unit").to_numpy() + outdataDF.get(outdataDF["Type"] == "Dealer").get("Expected shortage costs per time unit").to_numpy()

outdataDF.loc[outdataDF["Type"] == "RDC", "Expected holding costs per time unit"] = h_rdc*outdataDF.get(outdataDF["Type"] == "RDC").get("Expected stock on hand").to_numpy()
outdataDF.loc[outdataDF["Type"] == "RDC", "Total expected costs"] = outdataDF.get(outdataDF["Type"] == "RDC").get("Expected holding costs per time unit").to_numpy()



In [11]:
excel_path_outdata = "/Users/AlexanderLarsson/documents/VSCode/outdata.xlsx"
outdataDF.to_excel(excel_path_outdata,sheet_name = "Outdata_latest_testrun")

outdataDF


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,...,"R, subbatches",Waiting time,Lead time,R optimal,Realized item fill rate,Expected stock on hand,Expected backorders,Expected holding costs per time unit,Expected shortage costs per time unit,Total expected costs
0,1,RDC,Johannesburg,1,40,1,NaN,NBD,10.0,2.054649,...,-20.0,NaN,1.000000,-20.0,NaN,7.004688,12.004671,7.004688,NaN,7.004688
1,2,Dealer,Deal1,1,4,1,0.95,Poisson,1.0,NaN,...,NaN,1.200467,11.200467,15.0,0.964357,9.351639,0.052106,9.351639,0.990016,10.341655
2,3,Dealer,Deal2,3,25,1,0.95,Poisson,1.0,NaN,...,NaN,1.200467,4.200467,6.0,0.954883,7.345444,0.045911,7.345444,0.872310,8.217754
3,4,Dealer,Deal3,2,11,1,0.95,Poisson,1.0,NaN,...,NaN,1.200467,3.200467,5.0,0.953542,7.344224,0.044691,7.344224,0.849122,8.193345
4,5,Dealer,Deal4,1,37,1,0.95,Poisson,1.0,NaN,...,NaN,1.200467,2.200467,4.0,0.954975,7.337466,0.037933,7.337466,0.720729,8.058195
5,6,Dealer,Deal5,4,1,1,0.95,Poisson,1.0,NaN,...,NaN,1.200467,5.200467,9.0,0.952625,9.367099,0.067566,9.367099,1.283755,10.650854
6,7,Dealer,Deal6,3,3,1,0.95,Poisson,1.0,NaN,...,NaN,1.200467,4.200467,8.0,0.950467,9.367408,0.067875,9.367408,1.289628,10.657036
7,8,Dealer,Deal7,2,8,1,0.95,Poisson,1.0,NaN,...,NaN,1.200467,3.200467,7.0,0.950051,9.362393,0.062860,9.362393,1.194343,10.556736
8,9,Dealer,Deal8,5,45,1,0.95,Poisson,1.0,NaN,...,NaN,1.200467,6.200467,13.0,0.956806,12.379536,0.080003,12.379536,1.520059,13.899595
9,10,Dealer,Deal9,4,9,1,0.95,Poisson,1.0,NaN,...,NaN,1.200467,5.200467,12.0,0.954344,12.381584,0.082051,12.381584,1.558965,13.940549


In [12]:
print(f"Length: {len(rdc_f_u_probability_array)}, p(0) = {rdc_f_u_probability_array[0]}")

Length: 21, p(0) = 1.8851220974178386e-06


In [13]:
beta_arr

array([1.88512210e-06, 1.57117074e-05, 1.13391020e-04, 6.48278441e-04,
       2.93655295e-03, 1.05406584e-02, 2.99850600e-02, 6.76075465e-02,
       1.20830158e-01, 1.71187654e-01, 1.92266207e-01, 1.71187654e-01,
       1.20830158e-01, 6.76075465e-02, 2.99850600e-02, 1.05406584e-02,
       2.93655295e-03, 6.48278441e-04, 1.13391020e-04, 1.57117074e-05,
       1.72435192e-06])